# CPS Pickler
##### February 25, 2018 -- @bd_econ

In [1]:
# Import packages and identify datafiles
import re
import os
import struct
import pickle
import numpy as np
import pandas as pd
from calendar import month_abbr

# Location of data
os.chdir('E:/08_Other/Archive/')

In [2]:
# Information about dictionary start and end dates and how to parse the columns
d = {
 'January_2017_Record_Layout.txt': {'start': '2017-01-01', 'end': '2018-12-31', 'ixno': 3, 'split_string':'\t\t', 'p1': '\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\t+(.*)'},
 'January_2015_Record_Layout.txt': {'start': '2015-01-01', 'end': '2016-12-31', 'ixno': 3, 'split_string':'\t\t', 'p1': '\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\t+(.*)'},
 'January_2014_Record_Layout.txt': {'start': '2014-01-01', 'end': '2014-12-31', 'ixno': 3, 'split_string':'\t\t', 'p1': '\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\t+(.*)'},
 'January_2013_Record_Layout.txt': {'start': '2013-01-01', 'end': '2013-12-31', 'ixno': 3, 'split_string':'\t\t', 'p1': '\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\t+(.*)'},
 'may12dd.txt': {'start': '2012-05-01', 'end': '2012-12-31', 'ixno': 3, 'split_string':'\t\t', 'p1': '\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\t+(.*)'},
 'jan10dd.txt': {'start': '2010-01-01', 'end': '2012-04-30', 'ixno': 3, 'split_string':'    ', 'p1': '\n(?:\x0c)?(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\s+(.*)'},
 'jan09dd.txt': {'start': '2009-01-01', 'end': '2009-12-31', 'ixno': 3, 'split_string':'    ', 'p1': '\n(?:\x0c)?(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\s+(.*)'},
 'jan07dd.txt': {'start': '2007-01-01', 'end': '2008-12-31', 'ixno': 3, 'split_string':'    ', 'p1': '\n(?:\x0c)?(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\s+(.*)'},
 'augnov05dd.txt': {'start': '2005-08-01', 'end': '2006-12-31', 'ixno': 3, 'split_string':'    ', 'p1': '\n(?:\x0c)?(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\s+(.*)'},
 'may04dd.txt': {'start': '2004-05-01', 'end': '2005-7-31', 'ixno': 3, 'split_string':'    ', 'p1': '\n(?:\x0c)?(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\s+(.*)'},
 'jan03dd.txt': {'start': '2003-01-01', 'end': '2004-04-30', 'ixno': 3, 'split_string':'    ', 'p1': '\n(?:\x0c)?(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\s+(.*)'},
 'jan98dd.asc': {'start': '1998-01-01', 'end': '2002-12-31', 'ixno': 2, 'split_string':'    ', 'p1': 'D (\w+)\s+(\d{1,2})\s+(\d+)\s+', 'p2': 'D {var[0]}.*?(V .*?)\n(?:\x0c)?D ', 'p3': '(\d+.*?\d*)\s+(.*)'},
 'sep95_dec97_dd.txt': {'start': '1995-09-01', 'end': '1997-12-31', 'ixno': 3, 'split_string':'    ', 'p1': '\n(?:\x0c)?(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)', 'p2': '.*?VALID ENTRIES\n\n\s+(.*?)\n(?:\x0c)?\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', 'p3': '(\d+.*?\d*)\s+(.*)'}
}

In [3]:
# Series of interest 
s3 = ['PWORWGT', 'PWCMPWGT', 'HRHHID2', 'PRERNWA', 'PTERNWA', 'PWSSWGT']

s2 = ['PEHRUSL1', 'HRYEAR', 'HRYEAR4', 'PRUNEDUR', 'PRERNHLY', 'PTERNHLY']

# These series can be stored as categorical later on
s1 = ['HRMONTH', 'PESEX', 'PEMLR', 'PENLFRET', 'PENLFACT', 'PRDISC', 'GESTFIPS',
      'HRMIS', 'PRCOW1', 'PRFTLF', 'PREMPNOT', 'PRCIVLF', 'PEJHRSN','PRSJMJ', 
      'PEEDUCA', 'PRWKSTAT', 'PRMJOCC1', 'GTMETSTA', 'GEMETSTA', 'PEDWWNTO',
      'PRUNTYPE', 'PRMJIND1', 'PERACE', 'PTDTRACE', 'PRDTRACE', 'PRORIGIN',
      'PRDTHSP', 'PRCHLD', 'PRTAGE', 'PEAGE', 'PULINENO']   
s = s1 + s2 + s3 + ['HRHHID']

def text_repl(string_item):
    return (string_item.replace('PEAGE', 'PRTAGE').replace('PTERNHLY', 'PRERNHLY')
            .replace('PTERNWA', 'PRERNWA').replace('GEMETSTA', 'GTMETSTA')
            .replace('PERACE', 'PTDTRACE'))

In [4]:
# Build up dictionary with variable locations and codes
for ddi in d.items():
    dd = open(f'data/{ddi[0]}', 'r', encoding='iso-8859-1').read()
    p1 = re.compile(ddi[1]['p1'])
    vlist = [(text_repl(i[0]), int(i[ddi[1]['ixno']]), int(i[1])) 
             for i in p1.findall(dd) if i[0] in s]
    gclist = [(i[0], int(i[ddi[1]['ixno']]), int(i[1])) 
             for i in p1.findall(dd) if i[0] in s1]
    d[ddi[0]]['variables'] = {}
    d[ddi[0]]['vlist'] = vlist
    p3 = re.compile(ddi[1]['p3'])

    # Get the list of codes and their values for each coded variable
    for var in vlist:
        td = {}
        td['start'] = var[1]
        td['length'] = var[2]
        td['values'] = 'can be used directly'
        if var[0] in [g[0] for g in gclist]:
            p2 = re.compile(f'\n(?:\x0c)?{var[0]}{ddi[1]["p2"]}', 
                            re.MULTILINE|re.DOTALL)
            if ddi[0][-1] == 'c':
                p2 = re.compile(f'D {var[0]}.*?(V .*?)\n(?:\x0c)?D ', 
                                re.MULTILINE|re.DOTALL)
            vals = [[x.strip() for x in i.split('\n') if len(x.strip()) > 0] 
                    for i in p2.findall(dd)][0]
            val_list = [(i[0], i[1]) for i in 
                        [p3.findall(v)[0] for v in vals 
                         if len(p3.findall(v)) > 0]]
            td['values'] = val_list
            d[ddi[0]]['variables'][var[0]] = td

    # Special code to capture state codes, which are stored in two columns
    state_vals = []
    if ddi[0][-1] == 'c':
        d[ddi[0]]['variables']['GESTFIPS']['values'] = state_vals
    else:
        for state in d[ddi[0]]['variables']['GESTFIPS']['values']:
            if (' ' in state[1]) or ('\t' in state[1]):
                split_string = ddi[1]['split_string']
                stsplit = state[1].split(split_string)
                state1 = tuple([state[0], stsplit[0].strip()])
                state2 = tuple([int(stsplit[1].strip()[:2]), 
                                stsplit[1].strip()[-2:]])
                state_vals.append(state1)
                state_vals.append(state2)
            else:
                state_vals.append(state)
        d[ddi[0]]['variables']['GESTFIPS']['values'] = state_vals
        
with open('cps_dictionaries.pkl', 'wb') as handle:
    pickle.dump(d, handle)

In [5]:
# Set of functions for parsing raw data

# Use struct to read files faster 
def struct_constr(fieldspecs):
    """Specify which characters to retrieve and which to ignore"""
    unpack_len = 0
    unpack_fmt = ""
    for fieldspec in fieldspecs:
        start = fieldspec[1] - 1
        end = start + fieldspec[2]
        if start > unpack_len:
            unpack_fmt += str(start - unpack_len) + "x"
        unpack_fmt += str(end - start) + "s"
        unpack_len = end
    return struct.Struct(unpack_fmt).unpack_from

def fwf_to_list(file, unpacker, fieldspecs):
    """Return list of substrings"""
    fw = [i for i in fieldspecs if i[0] == 'PWSSWGT'][0]
    #Read monthly file and add it to annual dataframe
    return [tuple(map(int, unpacker(line))) 
            for line in open(f'data/{file}', 'rb') 
            if int(unpacker(line)[-1]) > 0]

# Convert list of lists to pandas df
def list_to_df(row_list, fieldspecs):
    """Store list as pandas dataframe"""
    df = (pd.DataFrame(row_list, columns=[v[0] for v in fieldspecs]))
    df[[s for s in s1 if s in df]] = df[[s for s in s1 if s in df]].astype(np.int8)    
    df[[s for s in s2 if s in df]] = df[[s for s in s2 if s in df]].astype(np.int16)
    df[[s for s in s3 if s in df]] = df[[s for s in s3 if s in df]].astype(np.int32)
#    id_vars = ['HRHHID', 'HRHHID2', 'PULINENO']
#    df['per_id'] = df['GESTFIPS'].astype(str).str.cat(
#        others=[df[i].astype(str) for i in id_vars if i in df.keys()])
    return df

# Manages the other functions
def monthly_to_annual(year, d, path):
    """Read monthly files and store as one annual file"""
    mo_list = [f for f in os.listdir('data/') 
               if f.endswith(f'{str(year)[2:]}pub.dat')]
    dt_list = [pd.to_datetime(f'{year}-{m[:3]}-01') for m in mo_list]
    dd_list = [i[0] for i in d.items() for date in dt_list 
               if date in pd.date_range(i[1]['start'], i[1]['end'])]

    files = list(zip(mo_list, dt_list, dd_list))
    df = pd.concat([list_to_df(
        fwf_to_list(f[0], struct_constr(d[f[2]]['vlist']), d[f[2]]['vlist']), 
        d[f[2]]['vlist']) for f in files]).reset_index(drop=True)
    df.to_feather(f'{path}cps_{year}.ft')

In [6]:
# Read data, year by year and pickle
path = 'C:/Working/econ_data/micro/data/'
for year in range(1996, 2019):
    monthly_to_annual(year, d, path)

In [7]:
d

{'January_2013_Record_Layout.txt': {'end': '2013-12-31',
  'ixno': 3,
  'p1': '\n(\\w+)\\s+(\\d+)\\s+(.*?)\t+.*?(\\d\\d*).*?(\\d\\d+)',
  'p2': '.*?VALID ENTRIES\n\n\\s+(.*?)\n(?:\x0c)?\\w+\\s+\\d+\\s+.*? \\s+.*?\\d\\d*.*?\\d\\d+',
  'p3': '(\\d+.*?\\d*)\t+(.*)',
  'split_string': '\t\t',
  'start': '2013-01-01',
  'variables': {'GESTFIPS': {'length': 2,
    'start': 93,
    'values': [('01', 'AL'),
     (30, 'MT'),
     ('02', 'AK'),
     (31, 'NE'),
     ('04', 'AZ'),
     (32, 'NV'),
     ('05', 'AR'),
     (33, 'NH'),
     ('06', 'CA'),
     (34, 'NJ'),
     ('08', 'CO'),
     (35, 'NM'),
     ('09', 'CT'),
     (36, 'NY'),
     ('10', 'DE'),
     (37, 'NC'),
     ('11', 'DC'),
     (38, 'ND'),
     ('12', 'FL'),
     (39, 'OH'),
     ('13', 'GA'),
     (40, 'OK'),
     ('15', 'HI'),
     (41, 'OR'),
     ('16', 'ID'),
     (42, 'PA'),
     ('17', 'IL'),
     (44, 'RI'),
     ('18', 'IN'),
     (45, 'SC'),
     ('19', 'IA'),
     (46, 'SD'),
     ('20', 'KS'),
     (47, 'TN'),
    

In [ ]:
df = pd.read_pickle(f'{path}/cps_2015.pkl')

In [ ]:
df[[s1 for s1 in s if s1 in df]] = df[[s1 for s1 in s if s1 in df]].astype('int32')

In [ ]:
s = ['PWORWGT', 'PWCMPWGT', 'HRHHID2', 
     'PRERNWA', 'PRERNHLY', 'PTERNWA', 'PTERNHLY', 'PRTAGE', 'PEAGE',
     'PRUNEDUR', 'PWSSWGT', 'HRYEAR', 'HRYEAR4']

In [ ]:
df[[s for s in s1 if s in df]]

In [ ]:
#df.to_feather(f'{path}cps_2015.ft')
pyarrow.feather.write_feather(df, f'{path}cps_2015.ft')

In [ ]:
df = pd.read_feather(f'{path}cps_2015.ft', nthreads=2)

In [ ]:
import numpy as np

In [ ]:
df2 = getattr(df, np.asscalar, lambda x=df: x)()

In [ ]:
df['HRHHID'] = df['HRHHID'].astype(np.int64)

In [ ]:
df['PRERNWA'].max()

In [ ]:
df.info(memory_usage='deep')

In [ ]:
import pyarrow

In [ ]:
pyarrow.feather.write_feather(df, f'{path}/cps_2015.ft')

In [ ]:
pickle.dumps(df, protocol=4)

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df2 = pd.read_pickle(f'{path}cps_2015.pkl')

In [ ]:
def fwf_to_list(file, unpacker, fieldspecs):
    """Return list of substrings"""
    fw = [i for i in fieldspecs if i[0] == 'PWSSWGT'][0]
    #Read monthly file and add it to annual dataframe
    return [tuple(map(int, unpacker(line))) 
            for line in open(f'data/{file}', 'rb') 
            if int(unpacker(line)[-1]) > 0]

In [ ]:
[i for i in dd if i[0] == 'PWSSWGT'][0]

In [ ]:
pd.read_pickle(f'{path}cps_2015.pkl')['GESTFIPS']

In [ ]:
fieldspecs = d['January_2017_Record_Layout.txt']['vlist']

fw = [i for i in fieldspecs if i[0] == 'PWSSWGT'][0]
[fw[1]:fw[1]+fw[2]]

In [ ]:
[fw[1]:fw[1]+fw[2]]

In [ ]:
d['January_2017_Record_Layout.txt']

In [ ]:
pd.read_pickle(f'{path}cps_2018.pkl.xz')['PTDTRACE']

In [ ]:
# Convert valid lines to list
def fwf_to_list(file, unpacker, fieldspecs):
    rows = []
    with open(f'data/{file}', 'rb') as f:
        for line in f:
            row = tuple(map(int, unpacker(line)))
            if row[-1] > 0:  # Filter out weightless rows
                rows.append(row)
    return rows

In [ ]:
# Manages the other functions
def monthly_to_annual(year, d, path):
    """Read monthly files and store as one annual file"""
    m_list = [f for f in os.listdir('data/') if f.endswith(f'{str(year)[2:]}pub.dat')]
    date = [pd.to_datetime(f'{year}-{m[:3]}-01') for m in m_list]
    data = pd.DataFrame()
    for m in m_list:
        date = pd.to_datetime(f'{year}-{m[:3]}-01')
        dd = ''
        for i in d.items():
            if (date >= pd.to_datetime(i[1]['start'])) and (date <= pd.to_datetime(i[1]['end'])):
                dd = i[0]
        
        #Read monthly file and add it to annual dataframe
        row_list = fwf_to_list(m, struct_constr(d[dd]['vlist']))
        df = list_to_df(row_list, d[dd]['vlist'])
        data = data.append(df)
    # Pickle annual file
    data.to_pickle(f'{path}cps_{year}.pkl')

In [ ]:
df

In [ ]:
import numpy as np

In [ ]:
path = 'C:/Working/econ_data/micro/data/'
df = pd.read_pickle(f'{path}cps_2014.pkl.xz', compression='xz')
df = df[(df['PRERNHLY'] > 212)]
df2 = pd.read_pickle(f'{path}cps_2015.pkl.xz', compression='xz')
df2 = df2[(df2['PRERNHLY'] > 212)]
df = df.append(df2)
df2 = pd.read_pickle(f'{path}cps_2016.pkl.xz', compression='xz')
df2 = df2[(df2['PRERNHLY'] > 212)]
df = df.append(df2)
df2 = pd.read_pickle(f'{path}cps_2017.pkl.xz', compression='xz')
df2 = df2[(df2['PRERNHLY'] > 212)]
df = df.append(df2)
df2 = pd.read_pickle(f'{path}cps_2018.pkl.xz', compression='xz')
df2 = df2[(df2['PRERNHLY'] > 212)]
df = df.append(df2)

In [ ]:
dft = df.set_index(['per_id', 'HRMIS']).sort_index()

In [ ]:
dft = dft.groupby(level=0).filter(
    lambda dft:np.in1d([4, 8], dft.index.get_level_values(1)).all()
)

In [ ]:
dft

In [ ]:
dft.groupby(level=0)

In [ ]:
d = []

for id in d:
    print(dft.loc[id])

In [ ]:
dft.index.get_level_values(0).unique()

In [ ]:
pay = [tuple([dft.loc[p_id, 4]['PRERNHLY'], 
       dft.loc[p_id, 8]['PRERNHLY']]) for p_id in 
       dft.index.get_level_values(0).unique() if 
       (dft.loc[p_id, 4]['PTDTRACE'] == dft.loc[p_id, 8]['PTDTRACE']) & 
       (dft.loc[p_id, 4]['PESEX'] == dft.loc[p_id, 8]['PTDTRACE']) & 
       (dft.loc[p_id, 8]['PEEDUCA'] >= dft.loc[p_id, 4]['PEEDUCA']) &
       (dft.loc[p_id, 8]['PRTAGE'] >= dft.loc[p_id, 4]['PRTAGE']) &
       (dft.loc[p_id, 8]['PRTAGE'] <= dft.loc[p_id, 4]['PRTAGE'] + 2) &
       (dft.loc[p_id, 4]['PRMJOCC1'] == dft.loc[p_id, 8]['PRMJOCC1']) & 
       (dft.loc[p_id, 4]['PRMJIND1'] == dft.loc[p_id, 8]['PRMJIND1'])]

In [ ]:
len([i for i in [round(o[1]) - round(o[0]) for o in pay] if i == 0])
#len(ind)PRTAGE

In [ ]:
7883.0/18271.0

In [ ]:
d = ['10100002103071002910012', '101000315145998030112']

In [ ]:
data = pd.DataFrame()

In [ ]:
for id in list(dft.index.get_level_values(0).unique()):
    dt = dft.loc[id]
    if (dt['PRMJIND1'].diff().sum() == 0) and (dt['PRMJOCC1'].diff().sum()== 0):
        data.at[id, pd.to_datetime(f"{dt.loc[8]['HRYEAR4']}-{dt.loc[8]['HRMONTH']}-01")] = dt['PRERNHLY'].diff().sum()
    #if abs(sum(dt['PRMJIND1'].diff() + dt['PRMJOCC1'].diff())) == 0:
        #data.at[id] = dt

In [ ]:
data

In [ ]:
f"{dt.loc[8]['HRYEAR4']}-{dt.loc[8]['HRMONTH']}-01"

In [ ]:
id = '10100002103071002910012'
if (dft.loc[id]['PRMJIND1'].diff().sum() == 0) and (dft.loc[id]['PRMJOCC1'].diff().sum()== 0):
    print('yes')

In [ ]:
dft.loc[id]['PRMJIND1'].diff().sum()

In [ ]:
list(dft.index.get_level_values(0).unique())

In [ ]:
df['diffs'] = df.groupby('per_id')['PRERNHLY'].transform(pd.Series.diff)

In [ ]:
df[df['diffs'].notnull()]

In [ ]:
import pickle

with open('cps_dictionaries.pkl', 'wb') as handle:
    pickle.dump(d, handle)

In [ ]:
with open('cps_dictionaries.pkl', 'rb') as handle:
    b = pickle.load(handle)

In [ ]:
b

In [ ]:
d

In [ ]:
dd

In [ ]:
for year in range(1996, 2019):
    m_list = [f for f in os.listdir('data/') if f.endswith(f'{str(year)[2:]}pub.dat')]
    for m in m_list:
        date = pd.to_datetime(f'{year}-{m[:3]}-01')
        for i in d.items():
            if (date >= pd.to_datetime(i[1]['start'])) and (date <= pd.to_datetime(i[1]['end'])):
                dd = i[0]
        
        
        
        
        #date = 
    #months = [pd.to_datetime(f'{year}-{m[:3]}-01') for m in m_list]
    #print(m_list)

In [ ]:
os.listdir('data/')

In [ ]:
[re.compile(dd[1]['p1']) for dd in dd_info.items()]

In [ ]:
dd_info['January_2013_Record_Layout.txt']['p1']

In [ ]:
d['sep95_dec97_dd.txt']['variables'].keys()

In [ ]:
vlist

In [ ]:
p2

In [ ]:
dd

In [ ]:
ddi[0].split('.')[0]

In [ ]:
p = re.compile('D (\w+)\s+(\d{1,2})\s+(\d+)\s+')
p3 = re.compile('(\d+.*?\d*)\s+(.*)')
#if (dd_name[-1] == 't') or (dd_name == 'may12dd'):
#    p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)')
#    p3 = re.compile('(\d+.*?\d*)\t+(.*)')
dd = open('data/jan98dd.asc', 'r', encoding='iso-8859-1').read()
vlist = [(text_repl(i[0]), int(i[2]), int(i[1])) 
         for i in p.findall(dd) if i[0] in s]
gclist = [(i[0], int(i[2]), int(i[1])) 
         for i in p.findall(dd) if i[0] in s2] 

In [ ]:
p.findall(dd)

In [ ]:
len(vlist)

In [ ]:
dd

In [ ]:
p

In [ ]:
d[dd_name] = {}
dd = open('data/jan98dd.asc', 'r', encoding='iso-8859-1').read()
p3 = re.compile('V.*?(\d+.*?\d*)\s+(.*)')

for var in vlist:
    d[dd_name][var[0]] = {}
    d[dd_name][var[0]]['start'] = var[1]
    d[dd_name][var[0]]['length'] = var[2]
    d[dd_name][var[0]]['values'] = 'can be used directly'
    if var[0] in [g[0] for g in gclist]:
        p2 = re.compile(f'D {var[0]}.*?(V .*?)\n(?:\x0c)?D ', re.MULTILINE|re.DOTALL)
        vals = [[x.strip() for x in i.split('\n') if len(x.strip()) > 0] for i in p2.findall(dd)][0]
        val_list = [(int(i[0]), i[1]) for i in [p3.findall(v)[0] for v in vals if len(p3.findall(v)) > 0]]
        d[dd_name][var[0]]['values'] = val_list

# Special code to capture state codes, which are stored in two columns
state_vals = []
if ddi[0][-1] == 'c':
    d[ddi[0]]['variables']['GESTFIPS']['values'] = state_vals
else:
    for state in d[dd_name]['GESTFIPS']['values']:
        if (' ' in state[1]) or ('\t' in state[1]):
            split_string = '    '
            if dd_name[-1] == 't':
                split_string = '\t\t'
            stsplit = state[1].split(split_string)
            state1 = tuple([state[0], stsplit[0].strip()])
            state2 = tuple([int(stsplit[1].strip()[:2]), stsplit[1].strip()[-2:]])
            state_vals.append(state1)
            state_vals.append(state2)
        else:
            state_vals.append(state)
    d[dd_name]['GESTFIPS']['values'] = state_vals

In [ ]:
dd

In [ ]:
dd

In [ ]:
state

In [ ]:
p3 = re.compile('(\d+.*?\d*)\t+(.*)')
[(int(i[0]), i[1]) for i in [p3.findall(v)[0] for v in vals if len(p3.findall(v)) > 0]]

In [ ]:
[p3.findall(v)[0] for v in vals]

In [ ]:
[[x.strip() for x in i.split('\n') if len(x.strip()) > 0] for i in p2.findall(dd)][0]

In [ ]:
len(vlist)

In [ ]:
len(d[dd_name])

In [ ]:
p2.findall(dd)

In [ ]:
d

In [ ]:
#dd_name = 'jan03dd'
dd_file = 'data/jan98dd.asc'
open(dd_file, 'r', encoding='iso-8859-1').read()

In [ ]:
print(open(dd_file, 'r', encoding='iso-8859-1').read())

In [ ]:
vals

In [ ]:
dd

In [ ]:
vlist

In [ ]:
d[dd_name].keys()

In [ ]:
d[dd_name].keys()

In [ ]:
d

In [ ]:
[g[0] for g in gclist]

In [ ]:
d['GESTFIPS']

In [ ]:
state_vals = []
for state in d['GESTFIPS']:
    if ' ' in state[1]:
        stsplit = state[1].split('     ')
        state1 = tuple([state[0], stsplit[0]])
        state2 = tuple([int(stsplit[1][:2]), stsplit[1][-2:]])
        state_vals.append(state1)
        state_vals.append(state2)
    else:
        state_vals.append(state)

In [ ]:
len(gclist)

In [ ]:
len(d)

In [ ]:
len(state_vals)

In [ ]:
p = re.compile('\n(\w+)\s+(\d+)\s+(.*?) \s+.*?(\d\d*).*?(\d\d+)')
dd = open(dd_file, 'r', encoding='iso-8859-1').read()
vlist = [(i[0], int(i[3]), int(i[1])) 
         for i in p.findall(dd) if i[0] in s2] 

print(len(vlist))

In [ ]:
dd_file = 'data/jan98dd.asc'
dd = open(dd_file, 'r', encoding='iso-8859-1').read()
var = 'PRMJIND1'
p2 = re.compile(f'{var}.*?VALID ENTRIES\n\n\s+(.*?)\n+\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', re.MULTILINE|re.DOTALL)
vals = [[x.strip() for x in i.split('\n') if len(x.strip()) > 0] for i in p2.findall(dd)][0]
p3 = re.compile('(\d+)\s+(.*)$')
val_list = [(int(i[0]), i[1]) for i in [p3.findall(v)[0] for v in vals]]
val_list

In [ ]:
val_list

In [ ]:
p2.findall(dd)

In [ ]:
vals = [[x.strip() for x in i.split('\n') if len(x.strip()) > 0] for i in p2.findall(dd)][0]
p3 = re.compile('(\d+)\s+([\w\s]+)')
val_list = [p3.findall(v) for v in vals]

In [ ]:
dd

In [ ]:
p3 = re.compile('(\d+)\s+([\w\s]+)')
val_list = [p3.findall(v) for v in vals]

In [ ]:
dd_file = 'data/jan07dd.txt'
dd = open(dd_file, 'r', encoding='iso-8859-1').read()
var = 'HRHHID'
p2 = re.compile(f'{var}.*?VALID ENTRIES\n\n\s+(.*?)\n+\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', re.MULTILINE|re.DOTALL)
vals = [[x.strip() for x in i.split('\n') if len(x.strip()) > 0] for i in p2.findall(dd)][0]
p3 = re.compile('(\d+)\s+(.*)$')
val_list = [(int(i[0]), i[1]) for i in [p3.findall(v)[0] for v in vals]]
val_list

In [ ]:
d = {}

dd_file = 'data/jan07dd.txt'
dd = open(dd_file, 'r', encoding='iso-8859-1').read()
#p2 = re.compile(f'{var}.*?VALID ENTRIES\n\n\s+(.*?)\n+\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', re.MULTILINE|re.DOTALL)
p3 = re.compile('(\d+.*?\d*)\s+(.*)$')
for var in gclist: 
    p2 = re.compile(f'\n{var[0]}.*?VALID ENTRIES\n\n\s+(.*?)\n+\w+\s+\d+\s+.*? \s+.*?\d\d*.*?\d\d+', re.MULTILINE|re.DOTALL)
    vals = [[x.strip() for x in i.split('\n') if len(x.strip()) > 0] for i in p2.findall(dd)][0]
    val_list = [(i[0], i[1]) for i in [p3.findall(v)[0] for v in vals]]
    d[var[0]] = val_list

In [ ]:
var

In [ ]:
d['GESTFIPS'][0]

In [ ]:
import sys # Check which version of python is being used
print(f'python {sys.version_info[0]}.{sys.version_info[1]}')
import pandas as pd    # Pandas to organize and make calcs
print(f'pandas {pd.__version__}')
import numpy as np     # Numpy for calculations
print(f'numpy {np.__version__}')
import os
import re
import struct
import wquantiles
import calendar
import sqlite3

In [ ]:
os.chdir('E:/08_Other/Archive/')
conn = sqlite3.connect('C:/Working/econ_data/micro/data/cps.db')

### Data Dictionaries

In [ ]:
# Series of interest 
s = ['PWORWGT', 'PWCMPWGT', 'HRHHID', 'HURESPLI', 'HRLONGLK', 'HRHHID2', 
     'PRERNWA', 'PRERNHLY', 'PRTAGE', 'PEAGE', 'PTERNWA', 'PTERNHLY',]

# These series can be stored as categorical later on
s2 = ['HRMONTH', 'PESEX', 'PEMLR', 'PENLFRET', 'PENLFACT', 'PRDISC', 'GESTFIPS',
      'HRMIS', 'PRCOW1', 'PRFTLF', 'PREMPNOT', 'PRCIVLF', 'PEJHRSN', 'PEMJOT',
      'PEEDUCA', 'PRWKSTAT', 'PRDTOCC1', 'GTMETSTA', 'GEMETSTA']   

s = s + s2
d = {}

p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\t+.*?(\d\d*).*?(\d\d+)')
for y in [2013, 2014, 2015, 2017]:
    # Read the data dictionary file to get the column names and locations
    dd_txt = f'data/January_{y}_Record_Layout.txt'
    dd = open(dd_txt, 'r', encoding='iso-8859-1').read()
    # Regular expression for info of interest based on pattern p
    d[y] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\s+.*?(\d\d*).*?(\d\d+)')
for v in ['augnov05', 'jan07', 'jan09', 'jan10', 'may12']:
    dd = open(f'data/{v}dd.txt', 'r', encoding='iso-8859-1').read()
    d[v] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
    # Manual touch up of data dictionaries
    d[v][5] = ('HRHHID2', 71, 5)
    d[v][8] = ('PRTAGE', 122, 2)
    d[v][20] = ('PRDTOCC1', 476, 2)

# May 2004 to July 2005 dictionary
dd = open(f'data/may04dd.txt', 'r', encoding='iso-8859-1').read()
d['may04'] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
d['may04'][7] = ('PRTAGE', 122, 2)   
d['may04'][19] = ('PRDTOCC1', 476, 2)
    
# 2003 to May 2004 dictionary    
dd = open(f'data/jan03dd.txt', 'r', encoding='iso-8859-1').read()
d['jan03'] = [(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]
d['jan03'][6] = ('GTMETSTA', 105, 1)
d['jan03'][7] = ('PRTAGE', 122, 2) 
d['jan03'][19] = ('PRDTOCC1', 476, 2)
    
# 1998 to 2002 dictionary    
p = re.compile('D (\w+)\s+(\d{1,2})\s+(\d+)\s+')
dd = open(f'data/jan98dd.asc', 'r', encoding='iso-8859-1').read()
d['jan98'] = [(i[0], int(i[2]), int(i[1])) for i in p.findall(dd) if i[0] in s]
d['jan98'][20] = ('PRERNHLY', 520, 4)
d['jan98'][21] = ('PRERNWA', 527, 8)
d['jan98'][6] = ('GTMETSTA', 105, 1)

# Manually note years where data dictionary does not change
d[2016] = d[2015]
d[2018] = d[2017]
d[2010] = d['jan10']
d[2011] = d['jan10']
d[2009] = d['jan09']
d[2008] = d['jan07']
d[2007] = d['jan07']
d[2006] = d['augnov05']
d[2003] = d['jan03']
d[2002] = d['jan98']
d[2001] = d['jan98']
d[2000] = d['jan98']
d[1999] = d['jan98']
d[1998] = d['jan98']

In [ ]:
#p = re.compile('\n(\w+)\s+(\d+)\s+(.*?)\s+.*?(\d\d*).*?(\d\d+)')
#dd = open(f'data/may04dd.txt', 'r', encoding='iso-8859-1').read()
#[(i[0], int(i[3]), int(i[1])) for i in p.findall(dd) if i[0] in s]

In [ ]:
#len(d['may04'])

In [ ]:
print(dd)

### Read a data dictionary into the notebook

In [ ]:
#print(open(f'data/jan10dd.txt', 'r', encoding='iso-8859-1').read())

### Unzipper

In [ ]:
#from zipfile import ZipFile
#for file in [f for f in os.listdir('data/') if f.endswith('pub.zip')]:
#    with ZipFile(f'data/{file}', 'r') as zip_ref:
#        zip_ref.extractall('data/')

### Convert .cps to .dat

In [ ]:
#for file in [f for f in os.listdir('data/') if f.endswith('cps')]:
#    os.rename(f'data/{file}', f'data/{file[:-4]}.dat')

### Some tools

In [ ]:
def struct_constr(fieldspecs):
    """Build a fast way to read fixed width files"""
    unpack_len = 0
    unpack_fmt = ""
    for fieldspec in fieldspecs:
        start = fieldspec[1] - 1
        end = start + fieldspec[2]
        if start > unpack_len:
            unpack_fmt += str(start - unpack_len) + "x"
        unpack_fmt += str(end - start) + "s"
        unpack_len = end
    field_indices = range(len(fieldspecs))
    return struct.Struct(unpack_fmt).unpack_from

def fwf_to_list(file, unpacker, l):
    with open(f'data/{file}', 'r', encoding='utf-8') as f:
        for line in f:
            raw_fields = unpacker(line.encode())
            if int(raw_fields[-1]) > 0:   # Filter out rows with no weight
                l.append(tuple(int(x) for x in raw_fields))
        
                
def list_to_pickle(l, fieldspecs, year):
    """Store list as pickled file for fast reading later"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    df.to_pickle(f'data/cps_{year}.pkl')
    
def list_to_df(l, fieldspecs, year):
    """Store list as pandas dataframe"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    return df

def df_to_sql(df, year):
    """Send pandas df to sql database"""
    df.to_sql(f'cps_{year}', conn, if_exists='replace')
    
def list_to_sql(l, fieldspecs, year):
    """Store list into sql database"""
    df = pd.DataFrame(l, columns = [f[0] for f in fieldspecs]
                     ).apply(pd.to_numeric,downcast='unsigned')
    for col in [f[0] for f in fieldspecs if f in s2]: 
        df[col] = df[col].astype('category').cat.as_ordered()
    df.to_sql(f'cps_{year}', conn, if_exists='replace')

def special_years(year, fs1, fs2, sm):
    """Handle cases where dictionary is split in middle of year"""
    if year == year:
        fieldspecs = fs1
        unpacker = struct_constr(fieldspecs)

        l = []
        # Fill list with monthly data from each monthly file
        for file in [f'{calendar.month_name[i].lower()[:3]}{str(year)[2:]}pub.dat' for i in range(1,sm)]:
            fwf_to_list(file, unpacker, l)
        df2 = list_to_df(l, fieldspecs, year)
                        
        fieldspecs = fs2
        unpacker = struct_constr(fieldspecs)

        # Fill list with monthly data from each monthly file
        for file in [f'{calendar.month_name[i].lower()[:3]}{str(year)[2:]}pub.dat' for i in range(sm,13)]:
            fwf_to_list(file, unpacker, l)
        df3 = list_to_df(l, fieldspecs, year)
        df = df2.append(df3)
        df_to_sql(df, year)

### Read monthly data as annual dataframe

In [ ]:
def monthly_to_annual(year):
    """Read monthly files and store as one annual file"""
    # Field specs (field name, start pos (1-relative), len)
    if year != 2012 and year !=2005 and year !=2004: 
        fieldspecs = d[year]
        unpacker = struct_constr(fieldspecs)

        l = []
        # Fill list with monthly data from each monthly file
        for file in [f for f in os.listdir('data/') if f.endswith(f'{str(year)[-2:]}pub.dat')]:
            fwf_to_list(file, unpacker, l)
        list_to_sql(l, fieldspecs, year)
        
    if year == 2012: special_years(2012, d[2011], d['may12'], 5)
    if year == 2005: special_years(2005, d['may04'], d['augnov05'], 8)
    if year == 2004: special_years(2004, d['jan03'], d['may04'], 5)

### Applying the function to selected years

In [ ]:
for year in range(1998, 1999):
    monthly_to_annual(year)

### Benchmarking

In [ ]:
# Usual Weekly Earnings 2017 Q1

sql_exec = ("select PRERNWA, PWORWGT from cps_2017 "
            "where (HRMONTH between 1 and 3) and (PRERNWA > -1) "
            "and (PRTAGE >=16) and (PRFTLF = 1)")

df = pd.read_sql(sql_exec, conn)
print('2017 Q1 Usual Weekly Earnings: ${0:,.2f}'.format(
    wquantiles.median(df['PRERNWA'], df['PWORWGT']) / 100.0))

In [ ]:
# Usual Weekly Earnings 2017 Q1
df = pd.read_pickle(f'data/cps_2017.pkl')
df = df[(df['PRERNWA'] > -1) & (df['PRTAGE'] >= 16) & 
        (df['PRFTLF'] == 1) & (df['HRMONTH'].isin([1, 2, 3]))]
        
print('2017 Q1 Usual Weekly Earnings: ${0:,.2f}'.format(
    wquantiles.median(df['PRERNWA'], df['PWORWGT']) / 100.0))

In [ ]:
# Prime age epop for women, Jan 2017 = 71.3 (LNU02300062)
df = pd.read_pickle(f'data/cps_2008.pkl')
df = df[(df['HRMONTH'] == 1) & (df['PESEX'] == 2) 
         & (df['PRTAGE'].isin(range(25,55)))]
empl = np.where(df['PREMPNOT']==1, 1, 0)
epop = np.average(empl.astype(float), weights=df['PWCMPWGT']) * 100
print(f'January 2008: {round(epop, 1)}')

In [ ]:
# Number of people not in the labor force
df = pd.read_pickle(f'data/cps_2017.pkl')
wc = float(max(df['HRMONTH'].unique()) * 10000000)
print(f"{y}: {(df[df['PEMLR'] >= 5]['PWORWGT'].sum() / wc) :,.0f}")

In [ ]:
df = pd.read_pickle(f'data/cps_2003.pkl')
wc = float(max(df['HRMONTH'].unique()) * 10000000)
df[(df['PENLFACT'] == 3) & (df['PRTAGE'].isin(range(25,35)))].groupby('PESEX')['PWCMPWGT'].sum() / wc 

### Making some calculations

In [ ]:
for year in range(1998, 2019):
    df = pd.read_pickle(f'data/cps_{year}.pkl')
    df = df[df['PRTAGE'].isin(range(25,35))]
    tot = df['PWCMPWGT'].sum()
    empl = df[df['PREMPNOT'] == 1]['PWCMPWGT'].sum()
    epop = empl / tot
    print(f'{year}: {epop * 100: 3.1f}')

In [ ]:
for year in range(1998, 2019):
    for month in range(1, 13):
        ex = (f"select PREMPNOT, PWCMPWGT from cps_{year} "
              f"where (HRMONTH = {month}) and (PRTAGE between 25 and 34)")
        df = pd.read_sql(ex, conn)
        empl = np.where(df['PREMPNOT']==1, 1, 0)
        epop = np.average(empl.astype(float), weights=df['PWCMPWGT']) * 100
        print(f'{month} {year}: {round(epop, 1)}')

In [ ]:
pd.read_sql("SELECT * FROM cps_2005", conn)

In [ ]:
import pandas as pd

In [ ]:
pd.read_pickle('data/cps_2000.pkl')['GESTFIPS']